<a href="https://colab.research.google.com/github/raiaiaia/llm-code-review-clj/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Junção de valores de uma coluna do csv

Como o processamento do modelo default foi interropido por falta de energia, foi gerado dois csv's respostas que precisavam ser unificados em um csv com o resultado completo.

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/rag_model_output_1.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/rag_model_output_2.csv')

In [ ]:
df_1.head()

,pull_request_url,comment_id,diff_hunk,content,cleaned_diff_hunk,diff_lines,model_comment
0,https://api.github.com/repos/adamtornhill/code...,95127516,"@@ -10,52 +10,106 @@ ""/some/path => G1 /anothe...",The test cases are a bit too noisy. We need to...,"(def ^:const multi-regexp-group-spec\n""^/some/...",9,1. **Readability:** Consider using clearer v...
1,https://api.github.com/repos/anmonteiro/lumo/p...,103127359,"@@ -811,6 +811,19 @@ (deps/index-upstream-fo...",I had to do this because apparently in the rea...,;; Introspection\n\n(defn ^:export get-arglist...,10,1. **Naming**: Consider renaming `get-arglis...
2,https://api.github.com/repos/anmonteiro/lumo/p...,119037104,"@@ -133,22 +135,31 @@ export function processL...",Wouldn't `readline.clearLine` from the api als...,function stopReverseSearch(\n replSession: RE...,10,1. Consider using `session.isReverseSearch =...
3,https://api.github.com/repos/anmonteiro/lumo/p...,120193925,"@@ -806,6 +807,57 @@ ...",these don't need to be atoms. can you switch t...,;; Taken from planck eval implementation\n;; T...,6,1. Atom 'fn-index': Consider usingvolatile i...
4,https://api.github.com/repos/anmonteiro/lumo/p...,125171069,"@@ -902,6 +902,35 @@ (recur (rt/read...",please use `(.-url url+source)`. `aget` is onl...,"(defn get-data-readers*\n ""Returns the merged...",5,1. Regarding the function 'get-data- readers...


In [ ]:
df_2.head()

,pull_request_url,comment_id,diff_hunk,content,cleaned_diff_hunk,diff_lines,model_comment
0,https://api.github.com/repos/adamtornhill/code...,95127516,"@@ -10,52 +10,106 @@ ""/some/path => G1 /anothe...",The test cases are a bit too noisy. We need to...,"(def ^:const multi-regexp-group-spec\n""^/some/...",9,NaN
1,https://api.github.com/repos/anmonteiro/lumo/p...,103127359,"@@ -811,6 +811,19 @@ (deps/index-upstream-fo...",I had to do this because apparently in the rea...,;; Introspection\n\n(defn ^:export get-arglist...,10,NaN
2,https://api.github.com/repos/anmonteiro/lumo/p...,119037104,"@@ -133,22 +135,31 @@ export function processL...",Wouldn't `readline.clearLine` from the api als...,function stopReverseSearch(\n replSession: RE...,10,NaN
3,https://api.github.com/repos/anmonteiro/lumo/p...,120193925,"@@ -806,6 +807,57 @@ ...",these don't need to be atoms. can you switch t...,;; Taken from planck eval implementation\n;; T...,6,NaN
4,https://api.github.com/repos/anmonteiro/lumo/p...,125171069,"@@ -902,6 +902,35 @@ (recur (rt/read...",please use `(.-url url+source)`. `aget` is onl...,"(defn get-data-readers*\n ""Returns the merged...",5,NaN


In [ ]:
df_final = df_1.copy()

In [ ]:
print(df_final.shape)

(5105, 7)


In [ ]:
print(df_final.columns)

Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'model_comment'],
      dtype='object')


In [ ]:
df_final.loc[2650:, 'model_comment'] = df_2.loc[2650:, 'model_comment'].values

In [ ]:
print(df_final.columns)

Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'model_comment'],
      dtype='object')


In [ ]:
df_final.to_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/rag_model_output_final.csv')

#Junção das colunas model_comment e cos_sim

Junção das duas colunas de comentário e similaridade de ambas saídas do modelo mistral (default e rag) para facilitar a análise e plotagem de gráficos

In [ ]:
DEFAULT_FILE_PATH = '/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/similarity_default_model.csv'
RAG_FILE_PATH = '/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/similarity_rag_model.csv'

In [ ]:
df_default = pd.read_csv(DEFAULT_FILE_PATH)
df_rag = pd.read_csv(RAG_FILE_PATH)

In [ ]:
df_final = df_default.copy()

In [ ]:
print(df_final.shape)
print(df_final.columns)

(5105, 8)
Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'model_comment', 'cos_sim'],
      dtype='object')


In [ ]:
df_final = df_final.rename(columns={'model_comment': 'default_model_comment'})
df_final = df_final.rename(columns={'cos_sim':'default_cos_sim'})

In [ ]:
print(df_final.shape)
print(df_final.columns)

(5105, 8)
Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'default_model_comment',
       'default_cos_sim'],
      dtype='object')


In [ ]:
df_final['rag_model_comment'] = df_rag['model_comment']
df_final['rag_cos_sim'] = df_rag['cos_sim']

In [ ]:
print(df_final.shape)
print(df_final.columns)

(5105, 10)
Index(['pull_request_url', 'comment_id', 'diff_hunk', 'content',
       'cleaned_diff_hunk', 'diff_lines', 'default_model_comment',
       'default_cos_sim', 'rag_model_comment', 'rag_cos_sim'],
      dtype='object')


In [ ]:
df_final.to_csv('/content/drive/MyDrive/rayanne-llm-code-review-clj/Dataset/output/similarity_final.csv')